<a href="https://colab.research.google.com/github/N4tth/taller2-RN/blob/main/vanillaRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import os
import itertools
import pandas as pd
import kagglehub
import requests

# Character-level language modeling

In [28]:
#load dataset
dataset_path = kagglehub.dataset_download("new-york-city/nyc-dog-names")
print("Ruta del dataset descargado:", dataset_path)

for f in os.listdir(dataset_path):
    if f.endswith(".csv"):
        csv_path = os.path.join(dataset_path, f)
        break


df = pd.read_csv(csv_path)
print("Dataset cargado correctamente")

print("Shape:", df.shape)
print("Columnas:", df.columns.tolist())
print(df.head())

Ruta del dataset descargado: C:\Users\laura\.cache\kagglehub\datasets\new-york-city\nyc-dog-names\versions\1
Dataset cargado correctamente
Shape: (16220, 2)
Columnas: ['Row_Labels', 'Count_AnimalName']
  Row_Labels  Count_AnimalName
0          1                 1
1          2                 2
2      40804                 1
3      90201                 1
4      90203                 1


In [29]:
#Preprocessing
df.columns = df.columns.str.strip()
df.columns = df.columns.str.replace(r'[^a-zA-Z0-9_#]+', '', regex=True)
print("Columnas limpias: ", df.columns.tolist())

col_name = None
col_count = None

for c in df.columns:
  if "row" in c.lower():
      col_name = c
  if "count" in c.lower():
    col_count = c
if not col_name or not col_count:
  raise ValueError("No se encontraron columnas de nombres o conteo")
print(f"Usando columnas → Nombres: {col_name}, Conteos: {col_count}")

names = df[col_name].dropna().astype(str).tolist()
counts = df[col_count].fillna(1).astype(int).tolist()

clean_names = []
for name, count in zip(names, counts):
  name = name.lower().strip()
  name = re.sub(r'[^a-zñáéíóú ]', '', name)
  if 2 <= len(name) <=12:
    repetitions = min(count, 10)
    clean_names.extend([name] * repetitions)
print(f"total nombres limpios: {len(clean_names)}")
print("Ejemplos: ", clean_names[:10])
#Corpus
text = "\n".join(clean_names)
print(f"Longitud total del texto: {len(text)} carácteres")

#Tokenization
chars = sorted(list(set(text)))
char_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_char = {i: ch for i, ch in enumerate(chars)}
vocab_size = len(chars)

print(f"Num caracteres únicos: {vocab_size}")
print("Ejemplos vocabulario:", chars[:20])

def get_subset_text(text, size):
  if len(text) < size:
    size = len(text)
  return text[:size]


Columnas limpias:  ['Row_Labels', 'Count_AnimalName']
Usando columnas → Nombres: Row_Labels, Conteos: Count_AnimalName
total nombres limpios: 43391
Ejemplos:  ['march', 'march', 'april', 'april', 'april', 'april', 'april', 'april', 'april', 'april']
Longitud total del texto: 290009 carácteres
Num caracteres únicos: 33
Ejemplos vocabulario: ['\n', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's']


seq_length. Que es el número de neuronas recurrentes. 64, 128, 256
words. Tamaño del conjunto de datos. 5000, 20000, 50000
max_iter. Número de iteraciones/época. 30, 50, 80


In [30]:
#experiment configurations
seq_lengths = [64, 128, 256]
dataset_sizes = [5000, 20000, 50000]
max_iters = [30, 50, 80]
clip_options = [True, False]

experiments = list(itertools.product(seq_lengths, dataset_sizes, max_iters, clip_options))
results = []

# Word-level language modeling

In [31]:
#load dataset
URL = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
text_shakespeare = requests.get(URL).text
print(text_shakespeare[:500])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [32]:
#preprocessing
text_shakespeare = text_shakespeare.lower().replace('\r', ' ').replace('\n', ' ')
tokens_shakespeare = re.findall(r"[a-z]+'?[a-z]+|[a-z]+", text_shakespeare)
vocab = sorted(set(tokens_shakespeare))
word_to_int = {w: i for i, w in enumerate(vocab)}
int_to_word = {i: w for w, i in word_to_int.items()}
vocab_size = len(vocab)

print(f"Palabras totales: {len(tokens_shakespeare)}")
print(f"Palabras únicas (vocab_size): {vocab_size}")
print("Ejemplos de tokens:", tokens_shakespeare[:20])

encoded = [word_to_int[w] for w in tokens_shakespeare]
print("Ejemplos de secuencia codificada:", encoded[:20])

Palabras totales: 203839
Palabras únicas (vocab_size): 12373
Ejemplos de tokens: ['first', 'citizen', 'before', 'we', 'proceed', 'any', 'further', 'hear', 'me', 'speak', 'all', 'speak', 'speak', 'first', 'citizen', 'you', 'are', 'all', 'resolved', 'rather']
Ejemplos de secuencia codificada: [4080, 1879, 894, 11907, 8203, 431, 4464, 5043, 6603, 9946, 295, 9946, 9946, 4080, 1879, 12348, 509, 295, 8825, 8541]


In [33]:
#experiment configurations
seq_lengths_shakespeare = [64, 128, 256]
dataset_sizes_shakespeare = [5000, 20000, 50000]
max_iters_shakespeare = [30, 50, 80]
clip_options_shakespeare = [True, False]

experiments_shakespeare = list(itertools.product(seq_lengths_shakespeare, dataset_sizes_shakespeare, max_iters_shakespeare, clip_options_shakespeare))
results_shakespeare = []

# Vanilla Neural Network

In [34]:
#Vanilla neural network from https://lucyliu-ucsb.github.io/posts/Backpropagation-of-a-vanilla-RNN with modifications
class VanillaRNN:
    def __init__(self, n_x, n_h, seq_length, learning_rate):
        self.n_x = n_x
        self.n_h = n_h
        self.seq_length = seq_length
        self.learning_rate = learning_rate

        self.Wxh = np.random.randn(n_h, n_x) * 0.01
        self.Whh = np.random.randn(n_h, n_h) * 0.01
        self.Why = np.random.randn(n_x, n_h) * 0.01
        self.bh = np.zeros((n_h, 1))
        self.by = np.zeros((n_x, 1))


        self.mWxh = np.zeros_like(self.Wxh)
        self.mWhh = np.zeros_like(self.Whh)
        self.mWhy = np.zeros_like(self.Why)
        self.mbh = np.zeros_like(self.bh)
        self.mby = np.zeros_like(self.by)

    def forward_pass(self, inputs, targets, hprev):
        x, h, y, p = {}, {}, {}, {}
        h[-1] = np.copy(hprev)
        loss = 0

        for t in range(len(inputs)):
            x[t] = np.zeros((self.n_x, 1))
            x[t][inputs[t]] = 1
            h[t] = np.tanh(self.Wxh @ x[t] + self.Whh @ h[t-1] + self.bh)
            y[t] = self.Why @ h[t] + self.by
            p[t] = np.exp(y[t]) / np.sum(np.exp(y[t]))
            loss -= np.log(p[t][targets[t], 0])

        return loss, x, h, p

    def backpropagation(self, x, h, p, targets, clip=True):
        dWxh, dWhh, dWhy = np.zeros_like(self.Wxh), np.zeros_like(self.Whh), np.zeros_like(self.Why)
        dbh, dby = np.zeros_like(self.bh), np.zeros_like(self.by)
        dhnext = np.zeros_like(h[0])

        for t in reversed(range(self.seq_length)):
            dy = np.copy(p[t])
            dy[targets[t]] -= 1
            dWhy += dy @ h[t].T
            dby += dy
            dh = self.Why.T @ dy + dhnext
            dhraw = (1 - h[t] * h[t]) * dh
            dbh += dhraw
            dWxh += dhraw @ x[t].T
            dWhh += dhraw @ h[t-1].T
            dhnext = self.Whh.T @ dhraw

        # Clipping enabling. Since there's no interface, the parameter "clip" functions as a switch
        if clip:
            for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
                np.clip(dparam, -5, 5, out=dparam)

        return dWxh, dWhh, dWhy, dbh, dby

    def update_para(self, dWxh, dWhh, dWhy, dbh, dby):
        for param, dparam, mem in zip(
            ['Wxh', 'Whh', 'Why', 'bh', 'by'],
            [dWxh, dWhh, dWhy, dbh, dby],
            ['mWxh', 'mWhh', 'mWhy', 'mbh', 'mby']
        ):
            setattr(self, mem, getattr(self, mem) + dparam * dparam)
            setattr(self, param, getattr(self, param) -
                    self.learning_rate * dparam / np.sqrt(getattr(self, mem) + 1e-8))

    def train(self, text, char_to_int, int_to_char, max_iter=10000, clip=True):
        iter_num, pos = 0, 0
        loss_list, perplexity_list = [], []
        hprev = np.zeros((self.n_h, 1))
        loss = -np.log(1.0 / self.n_x) * self.seq_length

        while iter_num < max_iter:
            if pos + self.seq_length + 1 >= len(text) or iter_num == 0:
                hprev = np.zeros((self.n_h, 1))
                pos = 0

            inputs = [char_to_int[ch] for ch in text[pos:pos+self.seq_length]]
            targets = [char_to_int[ch] for ch in text[pos+1:pos+self.seq_length+1]]
            pos += self.seq_length

            # Forward pass
            loss, x, h, p = self.forward_pass(inputs, targets, hprev)
            loss_list.append(loss)
            #Perplexity
            perplexity_list.append(np.exp(loss / self.seq_length))

            # Backprop
            dWxh, dWhh, dWhy, dbh, dby = self.backpropagation(x, h, p, targets, clip=clip)
            self.update_para(dWxh, dWhh, dWhy, dbh, dby)
            hprev = h[self.seq_length - 1]

            if iter_num % 1000 == 0:
                print(f"Iteración {iter_num} - Loss: {loss:.3f} - Perplejidad: {np.exp(loss/self.seq_length):.3f}")
            iter_num += 1

        return loss_list, perplexity_list
    def make_sample(self, hprev, seed_ix, n):
        """
        sample a length n sequence from the model
        """
        x = np.zeros((self.n_x, 1))
        x[seed_ix] = 1
        ixes = []
        h = np.copy(hprev)

        for t in range(n):
            h = np.tanh(self.Wxh @ x + self.Whh @ h + self.bh)
            y = self.Why @ h + self.by
            p = np.exp(y) / np.sum(np.exp(y))
            ix = np.random.choice(range(self.n_x), p = p.ravel())
            x = np.zeros((self.n_x, 1))
            x[ix] = 1
            ixes.append(ix)
        return ixes

    #Graphs
    def plot_loss(self, loss_list):
        plt.figure(figsize=(8, 5))
        plt.plot(loss_list, label="Loss", color='blue')
        plt.title("Curva de pérdida vs iteraciones")
        plt.xlabel("Iteraciones")
        plt.ylabel("Loss")
        plt.legend()
        plt.grid(True)
        plt.show()

    def plot_perplexity(self, perplexity_list):
        plt.figure(figsize=(8, 5))
        plt.plot(perplexity_list, label="Perplejidad", color='orange')
        plt.title("Curva de perplejidad vs iteraciones")
        plt.xlabel("Iteraciones")
        plt.ylabel("Perplejidad")
        plt.legend()
        plt.grid(True)
        plt.show()



# Character-level language modeling experiments

In [35]:
for seq_length, dataset_size, max_iter, clip in experiments:
    print(f"\n=== Entrenando modelo ===")
    print(f"seq_length={seq_length}, dataset_size={dataset_size}, max_iter={max_iter}, clip={clip}")

    subset_text = get_subset_text(text, dataset_size)
    rnn = VanillaRNN(n_x=len(char_to_int), n_h=seq_length, seq_length=seq_length, learning_rate=0.1)

    # Entrenamiento
    loss_list, perplexity_list = rnn.train(subset_text, char_to_int, int_to_char, max_iter=max_iter, clip=clip)

    # Generar nombres
    hprev = np.zeros((rnn.n_h, 1))
    seed_ix = np.random.randint(0, len(char_to_int))
    sample_ix = rnn.make_sample(hprev, seed_ix, 10)
    generated = ''.join(int_to_char[ix] for ix in sample_ix)

    print(f"→ Nombres generados: {generated}")

    results.append({
        "seq_length": seq_length,
        "dataset_size": dataset_size,
        "max_iter": max_iter,
        "clip": clip,
        "generated": generated,
        "final_loss": loss_list[-1],
        "final_perplexity": perplexity_list[-1]
    })



=== Entrenando modelo ===
seq_length=64, dataset_size=5000, max_iter=30, clip=True
Iteración 0 - Loss: 223.797 - Perplejidad: 33.011
→ Nombres generados: naaddaerh


=== Entrenando modelo ===
seq_length=64, dataset_size=5000, max_iter=30, clip=False
Iteración 0 - Loss: 223.777 - Perplejidad: 33.000
→ Nombres generados: ktpqéwqxsé

=== Entrenando modelo ===
seq_length=64, dataset_size=5000, max_iter=50, clip=True
Iteración 0 - Loss: 223.795 - Perplejidad: 33.009
→ Nombres generados: 
a

a
a
li

=== Entrenando modelo ===
seq_length=64, dataset_size=5000, max_iter=50, clip=False
Iteración 0 - Loss: 223.755 - Perplejidad: 32.989
→ Nombres generados: oudeó
h
ai

=== Entrenando modelo ===
seq_length=64, dataset_size=5000, max_iter=80, clip=True
Iteración 0 - Loss: 223.740 - Perplejidad: 32.981
→ Nombres generados: uz
lle
axr

=== Entrenando modelo ===
seq_length=64, dataset_size=5000, max_iter=80, clip=False
Iteración 0 - Loss: 223.756 - Perplejidad: 32.989
→ Nombres generados: m
aep
abas



KeyboardInterrupt: 

# Word-level language modeling experiments

In [37]:
def get_subset_tokens(encoded, size):
    size = min(size, len(encoded))
    return encoded[:size]

def train_word(self, encoded, max_iter=10000, clip=True):
    iter_num, pos = 0, 0
    loss_list, perplexity_list = [], []
    hprev = np.zeros((self.n_h, 1))
    N = len(encoded)
    while iter_num < max_iter:
        if pos + self.seq_length + 1 >= N or iter_num == 0:
            hprev = np.zeros((self.n_h, 1)); pos = 0
        inputs  = encoded[pos:pos+self.seq_length]
        targets = encoded[pos+1:pos+self.seq_length+1]
        pos += self.seq_length
        loss, x, h, p = self.forward_pass(inputs, targets, hprev)
        loss_list.append(loss)
        perplexity_list.append(np.exp(loss / self.seq_length))
        dWxh, dWhh, dWhy, dbh, dby = self.backpropagation(x, h, p, targets, clip=clip)
        self.update_para(dWxh, dWhh, dWhy, dbh, dby)
        hprev = h[self.seq_length - 1]
        iter_num += 1
    return loss_list, perplexity_list

VanillaRNN.train_word = train_word

def make_sample_words(self, hprev, seed_ix, n):
    x = np.zeros((self.n_x, 1)); x[seed_ix] = 1
    ixes, h = [], np.copy(hprev)
    for _ in range(n):
        h = np.tanh(self.Wxh @ x + self.Whh @ h + self.bh)
        y = self.Why @ h + self.by
        y -= np.max(y)                    
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(self.n_x), p=p.ravel())
        x = np.zeros((self.n_x, 1)); x[ix] = 1
        ixes.append(ix)
    return ixes

VanillaRNN.make_sample_words = make_sample_words

for seq_lengths_shakespeare, dataset_sizes_shakespeare, max_iters_shakespeare, clip_options_shakespeare in experiments_shakespeare:
    print(f"\n=== Entrenando modelo (WORD-LEVEL) ===")
    print(f"seq_length={seq_lengths_shakespeare}, dataset_size(tokens)={dataset_sizes_shakespeare}, max_iter={max_iters_shakespeare}, clip={clip_options_shakespeare}")

    subset_encoded = get_subset_tokens(encoded, dataset_sizes_shakespeare)
    if len(subset_encoded) <= seq_lengths_shakespeare + 1:
        print("→ Saltado: dataset_size <= seq_length+1")
        results.append({
            "seq_length": seq_lengths_shakespeare, "dataset_size": dataset_sizes_shakespeare,
            "max_iter": max_iters_shakespeare, "clip": clip_options_shakespeare, "status": "skipped: dataset too small"
        })
        continue

    rnn = VanillaRNN(n_x=vocab_size, n_h=seq_lengths_shakespeare, seq_length=seq_lengths_shakespeare, learning_rate=0.05)

    loss_list, perplexity_list = rnn.train_word(subset_encoded, max_iter=max_iters_shakespeare, clip=clip_options_shakespeare)

    hprev = np.zeros((rnn.n_h, 1))
    seed_word = "the" if "the" in word_to_int else next(iter(vocab))
    seed_ix = word_to_int[seed_word]
    sample_ix = rnn.make_sample_words(hprev, seed_ix, n=10)  # 10 palabras
    generated = ' '.join(int_to_word[ix] for ix in sample_ix)

    print(f"→ Texto generado: {generated}")

    results.append({
        "seq_length": seq_lengths_shakespeare,
        "dataset_size": dataset_sizes_shakespeare,
        "max_iter": max_iters_shakespeare,
        "clip": clip_options_shakespeare,
        "generated": generated,
        "final_loss": float(loss_list[-1]),
        "final_perplexity": float(perplexity_list[-1])
    })



=== Entrenando modelo (WORD-LEVEL) ===
seq_length=64, dataset_size(tokens)=5000, max_iter=30, clip=True
→ Texto generado: judases thickest that the swims the fellow famish not the

=== Entrenando modelo (WORD-LEVEL) ===
seq_length=64, dataset_size(tokens)=5000, max_iter=30, clip=False
→ Texto generado: shoes florizel what are true vi for they kings which

=== Entrenando modelo (WORD-LEVEL) ===
seq_length=64, dataset_size(tokens)=5000, max_iter=50, clip=True
→ Texto generado: obstacles proportionable injured or forth he it proud seat capitol

=== Entrenando modelo (WORD-LEVEL) ===
seq_length=64, dataset_size(tokens)=5000, max_iter=50, clip=False
→ Texto generado: flask righteous mere we serpent's mixture in absence unstable is

=== Entrenando modelo (WORD-LEVEL) ===
seq_length=64, dataset_size(tokens)=5000, max_iter=80, clip=True
→ Texto generado: mary's nice much tongue less us made them liking to

=== Entrenando modelo (WORD-LEVEL) ===
seq_length=64, dataset_size(tokens)=5000, max_it

KeyboardInterrupt: 

NOTA: Para obtener sólo las gráficas para el punto 1, correr sólo las celdas: imports, classVanillaRNN y with open

In [38]:
#Change the path. This was coded in colab
with open('/content/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))
char_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_char = {i: ch for i, ch in enumerate(chars)}
vocab_size = len(chars)

rnn = VanillaRNN(n_x=vocab_size, n_h=100, seq_length=25, learning_rate=1e-1)
loss_list, perplexity_list = rnn.train(text, char_to_int, int_to_char, max_iter=5000, clip=True)


rnn.plot_loss(loss_list)
rnn.plot_perplexity(perplexity_list)

FileNotFoundError: [Errno 2] No such file or directory: '/content/input.txt'

This is the content used in input.txt:

kmmatody: nomels bake tho pav.

M:
Atw: and I; thou onsel swere, lo! meroses ssseme noke shy ust but ker, woncter id imire ghy.

What Thes hereth:
Iss:
Drou wort, netesteme here to whont toy,
All My